<a href="https://colab.research.google.com/github/zambbo/BERT-blog-sentiment-analysis/blob/master/scoring_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
device = torch.device('cuda:0')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
    
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    
    def __len__(self):
        return len(self.labels)

In [ ]:
class BERTClassifier_sent(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier_sent, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
        
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
class BERTClassifier_subj(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier_subj, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
        
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

In [ ]:
sentiment_model = BERTClassifier_sent(bertmodel, dr_rate=0.5).to(device)
subjective_model = BERTClassifier_subj(bertmodel, dr_rate=0.5).to(device)

In [ ]:
sent_path = '/content/drive/MyDrive/dataset/sentiment_model.pt'
subj_path = '/content/drive/MyDrive/dataset/subjetive_model.pt'

In [ ]:
sentiment_model.load_state_dict(torch.load(sent_path))
subjective_model.load_state_dict(torch.load(subj_path))

In [ ]:
sentiment_model.eval()
subjective_model.eval()

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/dataset/미니어처.csv',index_col=0)
df.sample(3)

In [ ]:
test_df = df[:5]

In [ ]:
!pip install kss

In [ ]:
import re

def preprocessing_sentence(sentence):
    sent = re.sub("[^ ㄱ-ㅎ가-힣a-zA-Z1-9]", "", sentence)
    return sent

In [ ]:
posts = [preprocessing_sentence(sent) for sent in test_df['Post'].values]
posts[0]

In [ ]:
import kss
post_sentences = []

for post in posts:
    sents = [sent for sent in kss.split_sentences(post)]
    post_sentences.append(sents)
post_sentences[0]

In [ ]:
def sentence2bertdataset(posts):
    ret_sentences = []
    for post in posts:
        ret_sentences.append([[sent,'1'] for sent in post])
    return ret_sentences

In [ ]:
new_post_sentences = sentence2bertdataset(post_sentences)

In [ ]:
new_post_sentences[0]

In [ ]:
max_len = 64
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
test_set = [BERTDataset(sentences, 0, 1, tok, max_len, True, False) for sentences in new_post_sentences]
test_input = [torch.utils.data.DataLoader(test_s, batch_size=1, num_workers=5) for test_s in test_set]

In [ ]:
def sentiment_analysis(output_tensor):
    output_tensor = output_tensor.cpu().detach().numpy()
    argmax = np.argmax(output_tensor)
    if argmax == 2:
        return '긍정'
    if argmax == 1:
        return '중립'
    if argmax == 0:
        return '부정'

In [ ]:
def subjective_analysis(output_tensor):
    output_tensor = output_tensor.cpu().detach().numpy()
    argmax = np.argmax(output_tensor)
    if argmax == 1:
        return '객관'
    if argmax == 0:
        return '주관'

In [ ]:
for test in test_set[0]:
    a,b,c,d = test
    print(a,b,c,d)
    break

In [ ]:
neutral = 0
positive = 0
negative = 0
objective = 0
subjective = 0
for post_idx, test_post in enumerate(test_set):
    print(f"{post_idx+1}-----------------------------------------------------------------------")
    if post_idx == 1: break 
    for sent_idx, test in enumerate(test_post):
        token_ids, valid_length, segment_ids, label = test
        token_ids = torch.Tensor([token_ids])
        segment_ids = torch.Tensor([segment_ids])
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = torch.IntTensor([valid_length.item()])
        sent_out = sentiment_model(token_ids, valid_length, segment_ids)
        subj_out = subjective_model(token_ids, valid_length, segment_ids)
        print(f"{post_sentences[post_idx][sent_idx]} {sentiment_analysis(sent_out)} {subjective_analysis(subj_out)}")
        sentiment_result = sentiment_analysis(sent_out)
        subjective_result = subjective_analysis(subj_out)
        if sentiment_result == '긍정':
            positive += 1
        if sentiment_result == '중립':
            neutral += 1
        if sentiment_result == '부정':
            negative += 1
        if subjective_result == '객관':
            objective += 1
        if subjective_result == '주관':
            subjective += 1
print(f"긍정:{positive} 중립:{neutral} 부정:{negative} 객관:{objective} 주관:{subjective}")

In [ ]:
for idx, (token_ids, valid_length, segment_ids, label) in enumerate(test_input[0]):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    print(token_ids)
    print(valid_length)
    print(type(valid_length))
    out = sentiment_model(token_ids,valid_length,segment_ids)
    print(sentiment_analysis(out))

In [ ]:
post_sentences[0]